# Model Analisis Perilaku Konsumen Megaline

# Konten <a id='contents'></a>

* [1 Gambaran Besar](#big_picture)
    * [1.1 Pendahuluan](#intro)
    * [1.2 Deskripsi Data](#data_description)
    * [1.3 Tujuan dan Langkah-Langkah Pengerjaan Projek](#goals_and_step)

* [2 Prapemrosesan Data](#data_preprocessing)
    * [2.1 Memuat Data](#load_data)
    * [2.2 Eksplorasi Data Awal](#initial_data_exploration)
    * [2.3 Kesimpulan Awal](#initial_summary)
    
* [3 Memisahkan Data menjadi 3 Set](#split_data)
    
* [4 Melatih Model](#model_train)

* [5 Memeriksa Kualitas Data Set](#check_model_accuracy)

* [6 Sanity Check](#sanity_check)

* [7 Kesimpulan](#summary)

## Gambaran Besar <a id='big_picture'></a>

### Pendahuluan <a id='intro'></a>

Operator seluler Megaline merasa tidak puas karena banyak pelanggan mereka yang masih menggunakan paket lama. Perusahaan tersebut ingin mengembangkan sebuah model yang dapat menganalisis perilaku konsumen dan merekomendasikan salah satu dari kedua paket terbaru Megaline: Smart atau Ultra.

Sebagai seorang Data Scientiest saya diminta untuk mengembangkan sebuah model yang mampu memilih paket dengan tepat. Pada proyek ini, ambang batas untuk tingkat accuracy-nya adalah 0,75. 

### Deskripsi Data <a id='data_description'></a>

- сalls: jumlah panggilan
- minutes: total durasi panggilan dalam satuan menit
- messages: jumlah pesan teks
- mb_used: traffic penggunaan internet dalam satuan MB
- is_ultra: paket untuk bulan yang sedang berjalan (Ultra - 1, Smart - 0)

### Tujuan dan Langkah-Langkah Pengerjaan Projek <a id='goals_and_step'></a>

**Tujuan dari proyek ini adalah membuat model untuk merekomendasikan paket kepada konsumen Megaline**

**Langkah yang akan saya lakukan**
1. Mempelajari data pada tabel
2. Memisahkan data sumber menjadi training set, validation set, dan test set.
3. Memeriksa kualitas model yang berbeda dengan mengubah hyperparameter-nya.
4. Memeriksa kualitas model dengan menggunakan test set.
5. Melakukan sanity check terhadap model.

## Pra Pemrosesan Data <a id='data_preprocessing'></a>

In [ ]:
# Muat semua library
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats as st 
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV

### Memuat Data <a id='load_data'></a>

In [ ]:
# Muat file data menjadi DataFrame
df = pd.read_csv('/datasets/users_behavior.csv')

### Eksplorasi Data Awal <a id='initial_data_exploration'></a>

In [ ]:
# Menampilkan sample data untuk melihat data secara sekilas
df.sample(10)

,calls,minutes,messages,mb_used,is_ultra
2298,64.0,475.68,20.0,8540.27,0
1966,58.0,428.86,47.0,24815.00,0
635,69.0,524.58,79.0,20765.07,0
3176,58.0,401.49,71.0,28074.34,1
1206,28.0,204.69,15.0,20081.81,0
722,114.0,887.08,112.0,22415.92,1
2733,94.0,596.13,63.0,20943.57,0
2174,92.0,603.40,40.0,7818.69,0
873,71.0,556.84,71.0,19869.21,0
3043,78.0,530.57,98.0,13666.81,0


In [ ]:
# Menampilkan informasi/rangkuman umum tentang DataFrame
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


In [ ]:
# Memampilkan nilai statistik dari kolom numerik
df.describe()

,calls,minutes,messages,mb_used,is_ultra
count,3214.000000,3214.000000,3214.000000,3214.000000,3214.000000
mean,63.038892,438.208787,38.281269,17207.673836,0.306472
std,33.236368,234.569872,36.148326,7570.968246,0.461100
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,40.000000,274.575000,9.000000,12491.902500,0.000000
50%,62.000000,430.600000,30.000000,16943.235000,0.000000
75%,82.000000,571.927500,57.000000,21424.700000,1.000000
max,244.000000,1632.060000,224.000000,49745.730000,1.000000


In [ ]:
# Melihat distribusi perilaku konsumen
df['is_ultra'].value_counts()/ len(df)

0    0.693528
1    0.306472
Name: is_ultra, dtype: float64

### Kesimpulan Awal <a id='initial_summary'></a>

**Insights:**
1. Data yang digunakan merupakan data bersih yang tidak menggandung data null dan kesalahan tipe data, sehingga tidak perlu dilakukan proses penanggulangan data
2. Distribusi perilaku konsumen untuk durasi panggilan, sms, dan penggunaan data merupakan distribusi miring kanan
3. 69% konsumen Megaline menggunakan paket Smart dan hanya sekitar 31% yang menggunakan paket Ultra

## Memisahkan Data menjadi 3 set <a id='split_data'></a>

In [ ]:
# Mendefininikan data features dan data target
features = df.drop(['is_ultra'], axis=1)
target = df['is_ultra']

In [ ]:
# Membagi dataframe menjadi 3 set
features_train, features_check, target_train, target_check = train_test_split(
    features, target, test_size = 0.4, random_state = 12345)

features_valid, features_test, target_valid, target_test = train_test_split(
    features_check, target_check, test_size = 0.5, random_state = 12345)

In [ ]:
print(features_train.shape)
print(features_valid.shape)
print(features_test.shape)
print(target_train.shape)
print(target_valid.shape)
print(target_test.shape)

(1928, 4)
(643, 4)
(643, 4)
(1928,)
(643,)
(643,)


## Melatih Model <a id='model_train'></a>

Dari ketiga jenis model yang dipelajari pada sprint ini (Decision Tree, Random Forest, dan Logistics Regresion) didapatkan bahwa **Random Forest** memiliki tingkat akurasi yang tinggi, sehingga pada projek kali ini saya akan menggunakan model Random Forest.

In [ ]:
# Membuat model
model = RandomForestClassifier(random_state=54321, n_estimators=5)

In [ ]:
# Melatih model
model.fit(features_train, target_train)

RandomForestClassifier(n_estimators=5, random_state=54321)

In [ ]:
# Melihat akurasi model
model.score(features_valid, target_valid)

0.744945567651633

Berdasarkan hasil dapat dilihat bahwa tingkat akurasinya masih berada dibawah dari ketentuan projek, yaitu 0.74. Sehingga saya akan mencoba melakukan Tuning Hyperparameter untuk meningkatkan akurasinya.

In [ ]:
# Tuning Hyperparameter
best_score = 0
best_est = 0

for est in range(1, 11):
    model = RandomForestClassifier(random_state=54321, n_estimators=est)
    model.fit(features_train, target_train)
    score = model.score(features_valid, target_valid)
    if score > best_score:
        best_score = score
        best_est = est

print("Accuracy model terbaik pada validation set (n_estimators = {}): {}".format(best_est, best_score))

Accuracy model terbaik pada validation set (n_estimators = 10): 0.7791601866251944


Berdasarkan hasil dapat dilihat bahwa tingkat akurasinya sudah diatas dari ketentuan projek, yaitu 0.77. Akan tetapi saya akan mencoba untuk melakukan tuning hyperparameter menggunakan cara lain untuk melihat apakah nilai akurasinya dapat ditingkatkan

In [ ]:
# tuning hyper parameter menggunakan GridSearchCV
n_estimators_range = np.arange(1,100,1)
param_grid = dict(n_estimators = n_estimators_range)

model = RandomForestClassifier(random_state=54321)

grid = GridSearchCV(estimator = model, 
                    param_grid = param_grid, 
                    cv = 5)

In [ ]:
# Membuat model menggunakan metode GridSearchCV
grid.fit(features_train, target_train)

GridSearchCV(cv=5, estimator=RandomForestClassifier(random_state=54321),
             param_grid={'n_estimators': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
       35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51,
       52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68,
       69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85,
       86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99])})

In [ ]:
# Menentukan nilai hyperparameter terbaik
print('The best parameteres are %s with a score of %0.2f' 
      % (grid.best_params_, grid.best_score_))

The best parameteres are {'n_estimators': 88} with a score of 0.81


Berdasarkan hasil dapat dilihat bahwa tingkat akurasinya memang meningkat dari 0.77 menjadi 0.81, tetapi didapatkan bahwa proses prediksinya memakan waktu yang lebih lama hanya untuk menaikan tingkat akurasinya sebesar 0.04

## Memeriksa Kualitas Data Set <a id='check_model_accuracy'></a>

In [ ]:
# Membuat model dengan n_estimators = 88
model = RandomForestClassifier(random_state=54321, n_estimators=88)
model.fit(features_train, target_train)
score = model.score(features_valid, target_valid)
score

0.7931570762052877

In [ ]:
# Melakukan pengecekan akurasi dengan n_estimators = 88
test_score = model.score(features_test, target_test)
test_score

0.7869362363919129

In [ ]:
# Membuat model dengan n_estimators = 10
model = RandomForestClassifier(random_state=54321, n_estimators=10)
model.fit(features_train, target_train)
score = model.score(features_valid, target_valid)
score

0.7791601866251944

In [ ]:
# Melakukan pengecekan akurasi dengan n_estimators = 10
test_score = model.score(features_test, target_test)
test_score

0.7822706065318819

Berdasarkan data ini dapat dilihat bahwa sepertinya pengujian menggunakan model dengan n = 88 mengalami overfitting, sehingga saya akan menggunakan model dengan n = 10

## Sanity Check <a id='sanity_check'></a>

In [ ]:
df[df['is_ultra'] == 1].head(3)

,calls,minutes,messages,mb_used,is_ultra
3,106.0,745.53,81.0,8437.39,1
6,57.0,431.64,20.0,3738.90,1
8,7.0,43.39,3.0,2538.67,1


In [ ]:
df[df['is_ultra'] != 1].head(3)

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0


In [ ]:
new_features = pd.DataFrame(
    [
        [106.0, 745.53, 81.0, 8437.39],
        [57.0, 431.64, 20.0, 3738.90],
        [7.0, 43.39, 3.0, 2538.67],
        [40.0, 311.90, 83.0, 19915.42],
        [85.0, 516.75, 56.0, 22696.96],
        [77.0, 467.66, 86.0, 21060.45]
    ],
    columns=features_train.columns,
)

model = RandomForestClassifier(random_state=54321, n_estimators=10)
model.fit(features_train, target_train)
answers = model.predict(new_features)
answers

array([1, 0, 0, 0, 0, 0])

In [ ]:
new_features = pd.DataFrame(
    [
        [106.0, 745.53, 81.0, 8437.39],
        [57.0, 431.64, 20.0, 3738.90],
        [7.0, 43.39, 3.0, 2538.67],
        [40.0, 311.90, 83.0, 19915.42],
        [85.0, 516.75, 56.0, 22696.96],
        [77.0, 467.66, 86.0, 21060.45]
    ],
    columns=features_train.columns,
)

model = RandomForestClassifier(random_state=54321, n_estimators=88)
model.fit(features_train, target_train)
answers = model.predict(new_features)
answers

array([1, 1, 0, 0, 0, 0])

Berdasarkan hasil dari Sanity Check dapat dilihat bahwa meskipun n=88 mengalami overfitting akurasi yang dimiliki oleh n-estimator tetap lebih tinggi.

## Kesimpulan <a id='summary'></a>

Pada projek ini saya membuat sebuah Machine Learning sederhana menggunakan model Random Forest karena dinilai memiliki tingkat akurasi paling tinggi jika dibandingkan dengan model Decision Tree dan Logistic Regresion. 

Awalnya model yang saya buat hanya memiliki akurasi sebesar 0.74, setelah melakukan hyperparameter tunning dengan menggunakan metode GridSearchCv didapatkan peningkatan akurasi hingga 0.81 dengan nilai n_estimators sebesar 88. Dengan n=88 didapatkan penurunan tingkat akurasi pada dataset test, tetapi pada hasil sanity check didapatkan bahwa nilai akurasi n=88 lebih baik dari pada n=10.